*Licensed under the MIT License.*

# Text Classification of SST-2 Sentences using a 3-Player Introspective Model

In [1]:
import sys
sys.path.append("../..")
import os
import json
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import scrapbook as sb

from interpret_text.experimental.introspective_rationale import IntrospectiveRationaleExplainer
from interpret_text.experimental.common.preprocessor.glove_preprocessor import GlovePreprocessor
from interpret_text.experimental.common.preprocessor.bert_preprocessor import BertPreprocessor
from interpret_text.experimental.common.model_config.introspective_rationale_model_config import IntrospectiveRationaleModelConfig
from interpret_text.experimental.widget import ExplanationDashboard

from notebooks.test_utils.utils_sst2 import load_sst2_pandas_df
from notebooks.test_utils.utils_data_shared import load_glove_embeddings

## Introduction
In this notebook, we train and evaluate a  [three-player explainer](http://people.csail.mit.edu/tommi/papers/YCZJ_EMNLP2019.pdf) model on a subset of the [SST-2](https://nlp.stanford.edu/sentiment/index.html/) dataset. To run this notebook, we used the SST-2 data files provided [here](https://github.com/AcademiaSinicaNLPLab/sentiment_dataset).

### Set parameters
Here we set some parameters that we use for our modeling task.

In [2]:
# if quick run true, skips over embedding, most of model training, and model evaluation; used to quickly test pipeline
QUICK_RUN = True
MODEL_TYPE = "RNN" # currently support either RNN, BERT, or a combination of RNN and BERT
CUDA = True

# data processing parameters
DATA_FOLDER = "../../../data/sst2"
LABEL_COL = "labels" 
TEXT_COL = "sentences"
token_count_thresh = 1
max_sentence_token_count = 70

# training procedure parameters
load_pretrained_model = False
pretrained_model_path = "../models/rnn.pth"
MODEL_SAVE_DIR = os.path.join("..", "models")
model_prefix = "sst2rnpmodel"

In [3]:
model_config = {
    "cuda": CUDA,
    "model_save_dir": MODEL_SAVE_DIR, 
    "model_prefix": model_prefix,
    "lr": 2e-4
}

if QUICK_RUN:
    model_config["save_best_model"] = False
    model_config["pretrain_cls"] = True
    model_config["num_epochs"] = 1

if MODEL_TYPE == "RNN":
    # (i.e. not using BERT), load pretrained glove embeddings
    if not QUICK_RUN:
        model_config["embedding_path"] = load_glove_embeddings(DATA_FOLDER)
    else:
        model_config["embedding_path"] = os.path.join(DATA_FOLDER, "")

## Read Dataset
We start by loading a subset of the data for training and testing.

In [4]:
train_data = load_sst2_pandas_df('train')
test_data = load_sst2_pandas_df('test')
all_data = pd.concat([train_data, test_data])
if QUICK_RUN:
    batch_size = 50
    train_data = train_data.head(batch_size)
    test_data = test_data.head(batch_size)
X_train = train_data[TEXT_COL]
X_test = test_data[TEXT_COL]

In [5]:
# get all unique labels
y_labels = all_data[LABEL_COL].unique()

In [6]:
model_config["labels"] = np.array(sorted(y_labels))
model_config["num_labels"] = len(y_labels)

## Tokenization and embedding
The data is then tokenized and embedded using glove embeddings.

In [7]:
if MODEL_TYPE == "RNN":
    preprocessor = GlovePreprocessor(token_count_thresh, max_sentence_token_count)
    preprocessor.build_vocab(all_data[TEXT_COL])
if MODEL_TYPE == "BERT":
    preprocessor = BertPreprocessor()

In [8]:
# append labels to tokenizer output
df_train = pd.concat([train_data[LABEL_COL], preprocessor.preprocess(X_train)], axis=1)
df_test = pd.concat([test_data[LABEL_COL], preprocessor.preprocess(X_test)], axis=1)

## Explainer
Then, we create the explainer and train it (or load a pretrained model).
The steps involved to set up the explainer: 
- Initialize explainer
- Setup preprocessor for the explainer
- Supply necessary model configurations to the explainer
- Load the explainer once all necessary modules are setup
- Fit/Train the explainer

In [9]:
explainer = IntrospectiveRationaleExplainer(classifier_type=MODEL_TYPE, cuda=CUDA)

In [10]:
explainer.set_preprocessor(preprocessor)

In [11]:
explainer.build_model_config(model_config)

In [12]:
explainer.load()

dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1


In [13]:
explainer.fit(df_train, df_test)

  0%|                                                                                                                                                                 | 0/10 [00:00<?, ?it/s]
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 10%|███████████████▎                                                                                                                                         | 1/10 [00:00<00:02,  4.22it/s]
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

We can test the explainer and measure its performance:

In [ ]:
if not QUICK_RUN:
    explainer.score(df_test)
    sparsity = explainer.model.avg_sparsity
    accuracy = explainer.model.avg_accuracy
    anti_accuracy = explainer.model.avg_anti_accuracy
    print("Test sparsity: ", sparsity)
    print("Test accuracy: ", accuracy, "% Anti-accuracy: ", anti_accuracy)
    
    # for testing
    sb.glue("sparsity", sparsity)
    sb.glue("accuracy", accuracy)
    sb.glue("anti_accuracy", anti_accuracy)

## Local importances
We can display the found local importances (the most and least important words for a given sentence):

In [ ]:
# Enter a sentence that needs to be interpreted
sentence = "Beautiful movie ; really good , the popcorn was bad"
s2 = "a beautiful and haunting examination of the stories we tell ourselves to make sense of the mundane horrors of the world."
s3 = "the premise is in extremely bad taste , and the film's supposed insights are so poorly executed and done that even a high school dropout taking his or her first psychology class could dismiss them ."
s4= "This is a super amazing movie with bad acting"

local_explanation = explainer.explain_local(s4)

## Visualize explanations
We can visualize local feature importances as a heatmap over words in the document and view importance values of individual words.

In [ ]:
explainer.visualize(local_explanation)

In [ ]:
ExplanationDashboard(local_explanation)